# Root finding by the Interpolate Truncate and Project Method



## Usage Example

In [1]:
from scipy import optimize
from itp import itp # local method

In [2]:
# Quadratic function

def f(x):
    return (x**2 - 1)

root = itp(f, 0, 2, full_output=True)
print(root)


root = optimize.bisect(f, 0, 2, full_output=True) 
print(root)

(0.9999999999999991,       converged: True
           flag: 'converged'
 function_calls: 10
     iterations: 8
           root: 0.9999999999999991)
(1.0,       converged: True
           flag: 'converged'
 function_calls: 3
     iterations: 1
           root: 1.0)


In [3]:
# Lambert function

def f(x):
    import math
    return (x*math.exp(x) -1)

root = itp(f, -1, 1)
print(root)

root = optimize.bisect(f, -1, 1, full_output=True) 
print(root)

0.5671432904088337
(0.5671432904109679,       converged: True
           flag: 'converged'
 function_calls: 42
     iterations: 40
           root: 0.5671432904109679)


## Testing

### Benchmark

In [4]:
import math
import numpy as np

# Lambert
def f_lambert(x): return (x*math.exp(x) -1)

# Trigonometric 1
def f_trig1(x): return math.tan(x - 0.1)


# Trigonometric 2
def f_trig2(x): return (math.sin(x) + 0.5)


# Polynomial 1
def f_poly1(x): return 4*pow(x,5) + pow(x,2) + 1


# Polynomial 2
def f_poly2(x): return x + pow(x,10) - 1


# Exponential
def f_exp(x): return pow(math.pi, x) - math.e


# Logarithmic
def f_log(x): return math.log(math.sqrt(pow(x - 10/9, 2)))


# Posynomial
def f_posy(x): return 1/3 + np.sign(x)*pow(abs(x),1/3)+pow(x,3)


# Weierstrass
def f_weiers(x): return 0.001 + sum([math.sin((math.pi * pow(i,3) * x)/2)/(math.pi*pow(i,3)) for i in range(1,11)])


# Polynomial fraction
def f_polyfrac(x): return (x+2/3)/(x+1.01)


# Polynomial 3
def f_poly3(x): return pow(x*pow(10,6) - 1, 3)


# Exponential polynomial
def f_exppoly(x): return math.exp(x)*(f_poly3(x))


# Tangent polynomial
def f_tanpoly(x): return pow(x - 1/3, 2) * math.atan(x - 1/3)


# Circles
def f_circle(x): return np.sign(3*x+1)*(1 - math.sqrt(1 - pow(3*x+1, 2)/81))


# Step
def f_step(x): return (x>((1-pow(10,6))/pow(10,6)))*((1+pow(10,6))/pow(10,6))-1


# Geometric
def f_geom(x): return 1/(21*x-1)*(x!=(1/21))


# Truncated polynomial
def f_truncpoly(x): return (pow(x/2, 2) + math.ceil(x/2) - 1/2)


# Staircase
def f_stair(x): return math.ceil(10*x - 1) + 1/2


# Noisy line
def f_noisy(x): return x + math.sin(x*pow(10,6))/10 + 0.001


# Warsaw
def f_warsaw(x): return (1 + math.sin(1/(x+1))) - 1 if (x>-1) else -1


# Sawtooth
def f_saw(x): return 202*x - 2*math.floor((2*x + 0.01)/(0.02)) - 0.1


# Sawtooth Cube
def f_sawcube(x): return pow(f_saw(x), 3)



benchmark = [f_lambert, f_trig1, f_trig2, f_poly1, f_poly2, f_exp, f_log, f_posy, 
             f_weiers, f_polyfrac, f_poly3, f_exppoly, f_tanpoly, f_circle, f_step,
             f_geom, f_truncpoly, f_stair, f_noisy, f_warsaw, f_saw, f_sawcube]


### Scipy methods


In [5]:
methods = [itp, optimize.brentq, optimize.brenth, optimize.ridder, optimize.bisect, optimize.toms748]

### Executions

In [6]:
import pandas as pd
from scipy.optimize import RootResults

results = []
for meth in methods:
    for bench in benchmark:
        try:
            root = meth(bench, -1, 1, xtol=10e-10, full_output=True)
        except:
            root = (None, RootResults(None, None, None, -2))
        results.append([meth.__name__, bench.__name__, root[0], root[1].function_calls])


experiment_results = pd.DataFrame(results, columns=['method', 'function', 'root', 'function_calls'])


### Number of fucntion calls

In [7]:
experiment_results.groupby('function', sort=False).apply(
    lambda df: pd.Series({f'{method}': df.loc[df['method'] == method]['function_calls'].values[0]
                          for method in experiment_results.method.unique()})
)

,itp,brentq,brenth,ridder,bisect,toms748
function,,,,,,
f_lambert,15,10,9,10,33,10
f_trig1,11,9,9,12,33,9
f_trig2,11,8,8,14,33,9
f_poly1,32,10,11,14,33,11
f_poly2,33,12,10,12,33,11
f_exp,10,8,8,10,33,8
f_log,33,10,9,12,33,10
f_posy,32,10,10,16,33,9
f_weiers,12,11,11,14,33,13


In [8]:
def color_ratio_red(val):
    if val > 0.9 and val <= 1:
        return 'color: red'
    else:
        color = 'blue' if val < 0.9 else 'black'
        return 'color: %s' % color
    

experiment_results.groupby('function', sort=False).apply(
    lambda df: pd.Series({f'{method}': df.loc[df['method'] == 'itp']['function_calls'].values[0]/
                          df.loc[df['method'] == method]['function_calls'].values[0]
                          for method in experiment_results.method.unique()[1:]})
).style.applymap(color_ratio_red)

,brentq,brenth,ridder,bisect,toms748
function,,,,,
f_lambert,1.500000,1.666667,1.500000,0.454545,1.500000
f_trig1,1.222222,1.222222,0.916667,0.333333,1.222222
f_trig2,1.375000,1.375000,0.785714,0.333333,1.222222
f_poly1,3.200000,2.909091,2.285714,0.969697,2.909091
f_poly2,2.750000,3.300000,2.750000,1.000000,3.000000
f_exp,1.250000,1.250000,1.000000,0.303030,1.250000
f_log,3.300000,3.666667,2.750000,1.000000,3.300000
f_posy,3.200000,3.200000,2.000000,0.969697,3.555556
f_weiers,1.090909,1.090909,0.857143,0.363636,0.923077


In [9]:
experiment_results.groupby('function', sort=False).apply(
    lambda df: pd.Series({f'{method}': df.loc[df['method'] == method]['root'].values[0]
                          for method in experiment_results.method.unique()})
)

,itp,brentq,brenth,ridder,bisect,toms748
function,,,,,,
f_lambert,5.671433e-01,5.671433e-01,5.671433e-01,5.671433e-01,5.671433e-01,5.671433e-01
f_trig1,1.000000e-01,1.000000e-01,1.000000e-01,1.000000e-01,1.000000e-01,1.000000e-01
f_trig2,-5.235988e-01,-5.235988e-01,-5.235988e-01,-5.235988e-01,-5.235988e-01,-5.235988e-01
f_poly1,-8.439146e-01,-8.439146e-01,-8.439146e-01,-8.439146e-01,-8.439146e-01,-8.439146e-01
f_poly2,8.350790e-01,8.350790e-01,8.350790e-01,8.350790e-01,8.350790e-01,8.350790e-01
f_exp,8.735685e-01,8.735685e-01,8.735685e-01,8.735685e-01,8.735685e-01,8.735685e-01
f_log,1.111111e-01,1.111111e-01,1.111111e-01,1.111111e-01,1.111111e-01,1.111111e-01
f_posy,-3.702013e-02,-3.702013e-02,-3.702013e-02,-3.702013e-02,-3.702013e-02,-3.702013e-02
f_weiers,-2.006550e-04,-2.006550e-04,-2.006550e-04,-2.006555e-04,-2.006544e-04,-2.006550e-04


In [10]:
pd.options.display.float_format = '{:.11f}'.format
expected_roots = [
    0.56714329040978387299996866221035554975381578718651250813513107922304579308668456669321944696175229456224295046916509266208757,
    0.1, 
    -0.5235987755982988730771072305465838140328615665625176368291574320513027343810348331046724708903528446870254362703011736642613, 
    -0.8439145686492662388225542717891087051075198020869250372009777880836045705749995742001326055672745209806882560052102645862173, 
    0.83507904272355904760914999232488651656284695582827147550180448432414839794820277563242678967432713933565282268614559475423370, 
    0.87356852683023186835397746476334273882072986617613914765231984243070583100596488423749371036124228507911203886452470685331191, 
    0.11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111108888635628062243090135976, 
    -0.0370201277078609292732616619505402076596100252526065583766981393778389338254525401132500106002777822681459353789723561187288, 
    -0.0002006550332058068624265537704467349407778031629578576310723121629280196061177642822609485702921790811950150949874542629050, 
    -0.6666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666761915615940189581851315, 
    1.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000039666234427060070982816552260e-6, 
    1.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000039666234427060070982816552260e-6, 
    0.33333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333332380843840598104181486847, 
    -0.3333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333238084384059810418148684, 
    -0.9999989999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999960333765572939929017183, 
    0.04761904761904761904761904761904761904761904761904761904761904761904761904761904761904761904761904762585111542429925584652252, 
    0,
    0
]

expected_retults = experiment_results.groupby('function', sort=False).apply(
        lambda df: pd.Series({f'{method}': df.loc[df['method'] == method]['root'].values[0]
                              for method in experiment_results.method.unique()})
    ).drop(index = ['f_noisy', 'f_warsaw', 'f_saw', 'f_sawcube'])
expected_retults.sub(expected_roots, axis=0).abs().round(11)

,itp,brentq,brenth,ridder,bisect,toms748
function,,,,,,
f_lambert,0.00000000000,0.00000000000,0.00000000004,0.00000000000,0.00000000011,0.00000000000
f_trig1,0.00000000042,0.00000000000,0.00000000000,0.00000000049,0.00000000056,0.00000000000
f_trig2,0.00000000000,0.00000000000,0.00000000000,0.00000000050,0.00000000060,0.00000000000
f_poly1,0.00000000035,0.00000000000,0.00000000001,0.00000000049,0.00000000071,0.00000000050
f_poly2,0.00000000022,0.00000000000,0.00000000000,0.00000000000,0.00000000045,0.00000000000
f_exp,0.00000000000,0.00000000019,0.00000000008,0.00000000049,0.00000000036,0.00000000000
f_log,0.00000000039,0.00000000000,0.00000000000,0.00000000049,0.00000000010,0.00000000000
f_posy,0.00000000071,0.00000000001,0.00000000000,0.00000000050,0.00000000042,0.00000000000
f_weiers,0.00000000000,0.00000000000,0.00000000000,0.00000000042,0.00000000065,0.00000000000
